In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import astropy.constants as c 
import astropy.units as u

from scipy.misc import derivative

In [30]:
KB = c.k_B.cgs.value
PI = np.pi
E_B = (4.52 * u.eV).cgs.value
H = c.h.cgs.value
HBAR = H/(2*PI)
W_VIB = 5.4e14
MOI = 9.1e-41
I = (13.6 * u.eV).cgs.value
A0 = c.a0.cgs.value
M_H = 1.67e-24
M_P = c.m_p.cgs.value
M_E = c.m_e.cgs.value

index = np.arange(0, 101, 1)

In [28]:
#H2 internal partition functions

def h2_elec(T):
    e_el = -E_B/(KB * T)
    return 1 + np.exp(e_el)

def rot_deg(T, j):
    e_rot = -(j * (j + 1) * HBAR ** 2 / (2 * MOI)) / (KB * T)
    return (2*j + 1) * np.exp(e_rot)

def vib_f(T, n):
    e = -(n + 0.5) * HBAR * W_VIB / (KB * T)
    return np.exp(e)

def parallel(T):

    z_vib = np.sum(vib_f(T, index))
    z_para = np.sum(rot_deg(T, index[0:100:2]))
    z_ortho = np.sum(3*rot_deg(T, index[1:100:2]))
            
    return z_vib, z_para + z_ortho

def h2_int(T):
    h2_vib, h2_rot = parallel(T)
    
    return h2_elec(T) * h2_rot * h2_vib

In [11]:
#HI internal partition function

def n_max(T, P):
    return np.sqrt((KB * T) ** (1/3) / (2 * A0 * P ** (1/3)))

def h1_exp(T, n):
    eh1 = -I * (1 - (1/(n**2)))/(KB * T)
    return n ** 2 * np.exp(eh1)

def h1_elec(T, P):
    n_m = n_max(T, P)
    z_sum = 0
    n_elec = np.arange(1, int(n_m)+1, 1)

    z_sum = np.sum(h1_exp(T, n_elec))

    return 4 * z_sum
            

In [23]:
def C(T, P):
    z_tot = h1_elec(T, P) ** 2 / h2_int(T)

    e_c = -E_B / (KB * T)
    
    return (T * KB / P) * (PI * KB * T/ H ** 2) ** (3/2) * (M_H ** (3/2)) * z_tot * np.exp(e_c)

def ion_frac(T, P):

    return (-C(T,P) + np.sqrt(C(T,P) ** 2 + (4 * C(T,P))))/2

In [24]:
def Ch11(T, P):
    e = -I / (KB * T)
    
    return (KB * T / P) * (2 * PI * KB * T / H ** 2) ** (3/2) * (M_P * M_E / M_H) ** (3/2) * (4 / h1_elec(T, P)) * np.exp(e)

def ion_h11(T, P):
    
    return (-Ch11(T,P) + np.sqrt(Ch11(T,P) ** 2 + (4 * Ch11(T,P))))/2

In [25]:
#H2 specific helmholtz free energy

def f_h2_noconst(P, xhI, xhII):

    def F(T):

        ln = (KB / (P * (1 - xhI - xhII))) * (4 * PI * M_H * KB * T / H ** 2) ** (3/2) * h2_int(T)
        return T * np.log(ln)
        
    return F


In [26]:
#HI specific helmholtz free energy

def f_hI_noconst(P, xhI):

    def F(T):

        ln = (KB / (P * (xhI))) * (2 * PI * M_H * KB * T / H ** 2) ** (3/2) * h1_elec(T, P)
        return T * np.log(ln)
        
    return F
    

In [27]:
#HII specific helmholtz free energy

def f_hII_noconst(P, xhII):

    def F(T):
        ln = (KB * 2 / (P * (xhII))) * (2 * PI * M_P * KB * T / H ** 2) ** (3/2)
        return T * np.log(ln)
        
    return F